In [1]:
import pandas as pd
import numpy as np
import warnings
import pickle
import matplotlib.pyplot as plt
from scipy import stats
import tensorflow as tf
import seaborn as sns
from pylab import rcParams
from sklearn.model_selection import train_test_split
from keras.models import Model, load_model
from keras.layers import Input, Dense,Flatten
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers
from keras.layers import Dropout,Conv1D, MaxPooling1D,UpSampling1D,Reshape,Activation,BatchNormalization,LeakyReLU
from keras.models import Sequential
warnings.filterwarnings("ignore")

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
tsa = pd.read_csv("./tsa.csv",header=None)

In [3]:

tsa.columns = ["time"]

In [4]:
pat = pd.read_csv("./moss_plos_one_data.csv")

In [5]:
new_pat = pd.concat([pat,tsa],axis=1)

In [6]:
b = new_pat[new_pat.y==True]
id_true = list(set(list(b.id)))

In [7]:
# true_patient = new_pat[new_pat.id.isin(id_true)]
# true_patient["num"] = 0
# true_patient.shape
p = new_pat[(new_pat["tte"]<=0)&(new_pat["tte"]>-0.5)]

In [8]:
true_patient = p[p["tte"].notnull()]

In [9]:
true_patient = true_patient[true_patient.id.isin(id_true)]

In [10]:
true_patient.head()

,id,age,race,svc,LOS,ICU_Pt_Days,Mort,n_evts,eid,y,...,s2.edrk,s8.edrk,s24.edrk,srr,dfa,cosen,lds,af,AF,time
26318,133,63.7,wht,Other,14,5,0,1,8.0,True,...,0.000244,-0.000119,-0.000012,0.026143,1.180124,-2.378111,1.004765,0.0,0.0,6187.0
26319,133,63.7,wht,Other,14,5,0,1,8.0,True,...,0.000206,-0.000106,-0.000010,0.037133,0.897787,-2.352599,0.901798,0.0,0.0,6202.0
26320,133,63.7,wht,Other,14,5,0,1,8.0,True,...,0.000150,-0.000094,-0.000009,0.044280,0.909218,-2.186083,1.141100,0.0,0.0,6217.0
26321,133,63.7,wht,Other,14,5,0,1,8.0,True,...,0.000114,-0.000083,-0.000008,0.038783,0.728632,-2.361748,1.011130,0.0,0.0,6232.0
26322,133,63.7,wht,Other,14,5,0,1,8.0,True,...,0.000172,-0.000068,-0.000007,0.025394,0.877550,-2.299310,1.553938,0.0,0.0,6247.0


In [11]:
true_patient["num"] = 1
true_patient["cond"] = 0
true_patient.cond[true_patient.y==True] =1

In [12]:
true_patient.head()

,id,age,race,svc,LOS,ICU_Pt_Days,Mort,n_evts,eid,y,...,s24.edrk,srr,dfa,cosen,lds,af,AF,time,num,cond
26318,133,63.7,wht,Other,14,5,0,1,8.0,True,...,-0.000012,0.026143,1.180124,-2.378111,1.004765,0.0,0.0,6187.0,1,1
26319,133,63.7,wht,Other,14,5,0,1,8.0,True,...,-0.000010,0.037133,0.897787,-2.352599,0.901798,0.0,0.0,6202.0,1,1
26320,133,63.7,wht,Other,14,5,0,1,8.0,True,...,-0.000009,0.044280,0.909218,-2.186083,1.141100,0.0,0.0,6217.0,1,1
26321,133,63.7,wht,Other,14,5,0,1,8.0,True,...,-0.000008,0.038783,0.728632,-2.361748,1.011130,0.0,0.0,6232.0,1,1
26322,133,63.7,wht,Other,14,5,0,1,8.0,True,...,-0.000007,0.025394,0.877550,-2.299310,1.553938,0.0,0.0,6247.0,1,1


In [13]:
true_patient["cum_sum"] = true_patient.groupby("id")["cond"].cumsum()

In [14]:
t_p = true_patient[(true_patient.cum_sum!=0)]

In [15]:
t_p["new_count"] = 1

In [16]:
filterPat = t_p.groupby("id")["new_count"].sum().to_frame().reset_index()
filterPat.head(10)

,id,new_count
0,133,40
1,187,40
2,191,4
3,206,40
4,282,40
5,295,23
6,307,80
7,348,39
8,414,6
9,427,40


In [17]:
t_p[t_p.id==133]

,id,age,race,svc,LOS,ICU_Pt_Days,Mort,n_evts,eid,y,...,dfa,cosen,lds,af,AF,time,num,cond,cum_sum,new_count
26318,133,63.7,wht,Other,14,5,0,1,8.0,True,...,1.180124,-2.378111,1.004765,0.0,0.0,6187.0,1,1,1,1
26319,133,63.7,wht,Other,14,5,0,1,8.0,True,...,0.897787,-2.352599,0.901798,0.0,0.0,6202.0,1,1,2,1
26320,133,63.7,wht,Other,14,5,0,1,8.0,True,...,0.909218,-2.186083,1.141100,0.0,0.0,6217.0,1,1,3,1
26321,133,63.7,wht,Other,14,5,0,1,8.0,True,...,0.728632,-2.361748,1.011130,0.0,0.0,6232.0,1,1,4,1
26322,133,63.7,wht,Other,14,5,0,1,8.0,True,...,0.877550,-2.299310,1.553938,0.0,0.0,6247.0,1,1,5,1
26323,133,63.7,wht,Other,14,5,0,1,8.0,True,...,1.272835,-2.252002,1.254278,0.0,0.0,6262.0,1,1,6,1
26324,133,63.7,wht,Other,14,5,0,1,8.0,True,...,1.345833,-2.319829,0.844162,0.0,0.0,6277.0,1,1,7,1
26325,133,63.7,wht,Other,14,5,0,1,8.0,True,...,1.270736,-2.281478,0.944826,0.0,0.0,6292.0,1,1,8,1
26326,133,63.7,wht,Other,14,5,0,1,8.0,True,...,0.712641,-2.249134,1.274229,0.0,0.0,6307.0,1,1,9,1
26327,133,63.7,wht,Other,14,5,0,1,8.0,True,...,0.781597,-2.361413,1.525636,0.0,0.0,6322.0,1,1,10,1


In [18]:
filtered_id =  list(filterPat.id[filterPat.new_count>=4*6])
new_df = t_p[t_p.id.isin(filtered_id)]

In [19]:
len(filtered_id)

251

In [20]:
# new_df2 = new_df.groupby("id").head(96)


In [21]:
# reverse_df = new_df2[::-1]
reverse_df = new_df[::-1]

In [22]:
# new_df2 = new_df.loc[new_df.y==True]

In [23]:
reverse_df.head()

,id,age,race,svc,LOS,ICU_Pt_Days,Mort,n_evts,eid,y,...,dfa,cosen,lds,af,AF,time,num,cond,cum_sum,new_count
2211188,8060,67.92,wht,Cardiology,26,8,0,1,619.0,True,...,0.631044,-2.231604,1.618099,0.0,0.0,7889.0,1,1,40,1
2211187,8060,67.92,wht,Cardiology,26,8,0,1,619.0,True,...,0.631044,-2.231604,1.618099,0.0,0.0,7874.0,1,1,39,1
2211186,8060,67.92,wht,Cardiology,26,8,0,1,619.0,True,...,0.631044,-2.231604,1.618099,0.0,0.0,7859.0,1,1,38,1
2211185,8060,67.92,wht,Cardiology,26,8,0,1,619.0,True,...,0.631044,-2.231604,1.618099,0.0,0.0,7844.0,1,1,37,1
2211184,8060,67.92,wht,Cardiology,26,8,0,1,619.0,True,...,0.631044,-2.231604,1.618099,0.0,0.0,7829.0,1,1,36,1


In [24]:
# reverse_df = new_df2[::-1]
# reverse_df_24 = new_df2.groupby("id").head(48)
reverse_df_24 = reverse_df.groupby("id").head(24)

In [25]:
reverse_df_24.shape

(6024, 67)

In [26]:
# reverse_df_12 = reverse_df_24.groupby("id").head(48)
reverse_df_24 = reverse_df_24[::-1]

In [27]:
reverse_df_24.head()

,id,age,race,svc,LOS,ICU_Pt_Days,Mort,n_evts,eid,y,...,dfa,cosen,lds,af,AF,time,num,cond,cum_sum,new_count
26334,133,63.7,wht,Other,14,5,0,1,8.0,True,...,1.075764,-2.262676,0.864124,0.0,0.0,6427.0,1,1,17,1
26335,133,63.7,wht,Other,14,5,0,1,8.0,True,...,1.071926,-2.331087,0.781297,0.0,0.0,6442.0,1,1,18,1
26336,133,63.7,wht,Other,14,5,0,1,8.0,True,...,0.928452,-2.275927,0.702950,0.0,0.0,6457.0,1,1,19,1
26337,133,63.7,wht,Other,14,5,0,1,8.0,True,...,0.775527,-2.342968,0.920636,0.0,0.0,6472.0,1,1,20,1
26338,133,63.7,wht,Other,14,5,0,1,8.0,True,...,1.002887,-2.261134,1.026540,0.0,0.0,6487.0,1,1,21,1


In [28]:
reverse_df_24["t2"] = reverse_df_24.time.shift(1)

In [29]:
reverse_df_24["diff"] = reverse_df_24.time - reverse_df_24["t2"]

In [30]:
reverse_df_24["diff"].fillna(15,inplace=True)


In [31]:
first_row = reverse_df_24.groupby("id").apply(lambda x: x.iloc[0:1]).reset_index(level=1).reset_index(drop=True)

In [32]:
first_row.rename(columns = {"level_1":"ind"},inplace=True)

In [33]:
first_row.head()

,ind,id,age,race,svc,LOS,ICU_Pt_Days,Mort,n_evts,eid,...,lds,af,AF,time,num,cond,cum_sum,new_count,t2,diff
0,26334,133,63.70,wht,Other,14,5,0,1,8.0,...,0.864124,0.0,0.0,6427.0,1,1,17,1,NaN,15.0
1,35966,187,52.23,wht,Cardiology,22,11,0,1,9.0,...,1.618099,0.0,0.0,6710.0,1,1,17,1,6772.0,-62.0
2,40649,206,55.34,blck,Other Medicine,9,2,0,1,11.0,...,1.808414,0.0,0.0,7021.0,1,1,17,1,7055.0,-34.0
3,55956,282,67.28,wht,Cardiothoracic Surgery,14,4,0,1,15.0,...,1.731868,0.0,0.0,6810.0,1,1,17,1,7366.0,-556.0
4,65619,307,47.69,wht,Cardiology,73,24,1,2,19.0,...,1.014304,0.0,0.0,99414.0,1,1,57,1,7155.0,92259.0


In [34]:
c1 = reverse_df_24.groupby("id").apply(lambda x: x.iloc[1:]).reset_index(level=1).reset_index(drop=True)

In [35]:
# c1.groupby("id")["diff"].max()
c1.rename(columns = {"level_1":"ind"},inplace=True)

In [36]:
max_diff = c1.groupby("id")["diff"].max().to_frame().reset_index()
lt_60_id  = max_diff[max_diff["diff"] <= 45].id

In [37]:
lt_60_id.shape #227 paitents left

(227,)

In [38]:
first_row_left  = first_row[first_row.id.isin(lt_60_id.tolist())]
c1_left = c1[c1.id.isin(lt_60_id.tolist())]

In [39]:
empty_df = pd.DataFrame()
i = 0 
for index, row in c1_left.iterrows():
    num = row["diff"]/15.0
    if num >1:
        empty_df= empty_df.append([c1_left.iloc[i-1,]]*int(num)) 
    i+=1


            
         
        

In [40]:
c1_left.shape

(5221, 70)

In [41]:
c_name = c1_left.columns

In [42]:
empty_df = empty_df[c_name]

In [43]:
c1_left.shape

(5221, 70)

In [44]:
merged_df1 = pd.concat([c1_left,first_row_left,empty_df],axis=0)

In [45]:
#sort the df by index col
merged_df1.sort_values("ind",inplace=True) #5515-227-5221 = 67 rows added

In [46]:
first_row_left.shape

(227, 70)

In [47]:
merged_df1.shape

(5515, 70)

In [48]:
merged_df2 = merged_df1.groupby("id").head(24)

In [49]:
df_12 = merged_df2.copy()
df_12.head()

,ind,id,age,race,svc,LOS,ICU_Pt_Days,Mort,n_evts,eid,...,lds,af,AF,time,num,cond,cum_sum,new_count,t2,diff
0,26334,133,63.7,wht,Other,14,5,0,1,8.0,...,0.864124,0.0,0.0,6427.0,1,1,17,1,NaN,15.0
0,26335,133,63.7,wht,Other,14,5,0,1,8.0,...,0.781297,0.0,0.0,6442.0,1,1,18,1,6427.0,15.0
1,26336,133,63.7,wht,Other,14,5,0,1,8.0,...,0.702950,0.0,0.0,6457.0,1,1,19,1,6442.0,15.0
2,26337,133,63.7,wht,Other,14,5,0,1,8.0,...,0.920636,0.0,0.0,6472.0,1,1,20,1,6457.0,15.0
3,26338,133,63.7,wht,Other,14,5,0,1,8.0,...,1.026540,0.0,0.0,6487.0,1,1,21,1,6472.0,15.0


In [50]:
id_list = list(set(pat['id']))  # 8105
id_false = [id for id in id_list if id not in id_true]
false_patient = pat[pat.id.isin(id_false)]

In [51]:
false_patient["cond"] = 1
false_count = false_patient.groupby("id")["cond"].sum().to_frame()
false_count = false_count[false_count.cond>24]

In [52]:
false_id = false_count.index

In [53]:
false_patient = pat[pat.id.isin(false_id)]
false_patient.head()

,id,age,race,svc,LOS,ICU_Pt_Days,Mort,n_evts,eid,y,...,edrk,s2.edrk,s8.edrk,s24.edrk,srr,dfa,cosen,lds,af,AF
0,1,39.35,wht,Cardiothoracic Surgery,4,1,0,0,NaN,False,...,23.527702,0.000002,0.000001,-4.918909e-07,0.027926,0.722903,-2.391665,1.670820,0.0,0.0
1,1,39.35,wht,Cardiothoracic Surgery,4,1,0,0,NaN,False,...,25.659513,0.002369,0.002369,2.368679e-03,0.031948,0.728738,-2.432732,1.995264,0.0,0.0
2,1,39.35,wht,Cardiothoracic Surgery,4,1,0,0,NaN,False,...,26.504228,0.001654,0.001654,1.653626e-03,0.023217,0.804216,-2.459413,2.058334,0.0,0.0
3,1,39.35,wht,Cardiothoracic Surgery,4,1,0,0,NaN,False,...,26.203203,0.000986,0.000986,9.856909e-04,0.010812,0.974394,-2.455503,1.986356,0.0,0.0
4,1,39.35,wht,Cardiothoracic Surgery,4,1,0,0,NaN,False,...,24.115443,0.000191,0.000191,1.910191e-04,0.007782,0.768893,-2.429776,1.900642,0.0,0.0


In [54]:
#randomly pick 12 hours 
#split training and test set
false_id = list(false_id)
true_id = list(set(df_12.id))
np.random.seed(200)
np.random.shuffle(list(false_id))
np.random.shuffle(list(true_id))


In [55]:
false_patient["count"] = false_patient.groupby("id").cumcount()


In [56]:
false_patient["count_1"] = false_patient["count"] - 24
# false_patient["count"] = false_patient.groupby("id")["count"].max()


In [57]:
max_num =  false_patient.groupby("id")["count_1"].max()


In [58]:
max_num = max_num.reset_index()
new_false_patient = pd.merge(false_patient,max_num,how="left",on="id")


In [59]:
new_false_patient.drop(["count_1_x"],axis=1,inplace=True)


In [60]:
import random
def agg(x):
    up = random.randint(0,x["count_1_y"].values[1])
    low = up+24
    
    return x.iloc[up:low,]

df3 = new_false_patient.groupby("id").apply(lambda x: agg(x))


In [61]:
df3 = df3.reset_index(drop=True)
df3.drop(["count","count_1_y"],axis=1,inplace=True)


In [62]:
df3.shape


(172776, 62)

In [63]:
df_12.head()

,ind,id,age,race,svc,LOS,ICU_Pt_Days,Mort,n_evts,eid,...,lds,af,AF,time,num,cond,cum_sum,new_count,t2,diff
0,26334,133,63.7,wht,Other,14,5,0,1,8.0,...,0.864124,0.0,0.0,6427.0,1,1,17,1,NaN,15.0
0,26335,133,63.7,wht,Other,14,5,0,1,8.0,...,0.781297,0.0,0.0,6442.0,1,1,18,1,6427.0,15.0
1,26336,133,63.7,wht,Other,14,5,0,1,8.0,...,0.702950,0.0,0.0,6457.0,1,1,19,1,6442.0,15.0
2,26337,133,63.7,wht,Other,14,5,0,1,8.0,...,0.920636,0.0,0.0,6472.0,1,1,20,1,6457.0,15.0
3,26338,133,63.7,wht,Other,14,5,0,1,8.0,...,1.026540,0.0,0.0,6487.0,1,1,21,1,6472.0,15.0


In [64]:
df_12.drop(["num","cond","ind","time","t2","diff","new_count","cum_sum"],axis=1,inplace=True)


In [65]:
total = pd.concat([df3,df_12],axis=0)



In [66]:
total.shape

(178224, 62)

In [67]:
total.head()

,id,age,race,svc,LOS,ICU_Pt_Days,Mort,n_evts,eid,y,...,edrk,s2.edrk,s8.edrk,s24.edrk,srr,dfa,cosen,lds,af,AF
0,1,39.35,wht,Cardiothoracic Surgery,4,1,0,0,NaN,False,...,18.293803,-0.000269,-0.000384,-0.000384,0.014371,0.721363,-2.506159,1.961073,0.0,0.0
1,1,39.35,wht,Cardiothoracic Surgery,4,1,0,0,NaN,False,...,18.850548,-0.000167,-0.000371,-0.000371,0.031302,0.631820,-2.465031,2.106385,0.0,0.0
2,1,39.35,wht,Cardiothoracic Surgery,4,1,0,0,NaN,False,...,19.134593,-0.000085,-0.000354,-0.000354,0.038697,0.522393,-2.466816,2.371036,0.0,0.0
3,1,39.35,wht,Cardiothoracic Surgery,4,1,0,0,NaN,False,...,19.070390,-0.000073,-0.000339,-0.000339,0.042782,0.581481,-2.420884,2.062371,0.0,0.0
4,1,39.35,wht,Cardiothoracic Surgery,4,1,0,0,NaN,False,...,19.406295,-0.000013,-0.000320,-0.000320,0.027743,0.618394,-2.458502,2.293772,0.0,0.0


In [69]:
ecg_col = ['hr', 's2.hr', 's8.hr', 's24.hr', 'n.edrk',
       'edrk', 's2.edrk', 's8.edrk', 's24.edrk', 'srr', 'dfa', 'cosen', 'lds',
       'af', 'AF']

In [67]:
z = ['hr']
num = 178224*len(z)

356448

In [68]:
d = pd.DataFrame(np.array(total[z]).reshape(7426,int(num/7426)))

In [69]:
d.head()

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,92.688889,90.442714,91.053393,91.269188,90.459399,91.191111,90.966667,90.596218,90.072303,88.948775,...,86.011136,83.482720,82.423634,81.283166,80.453229,80.997775,80.985507,79.795987,79.205784,79.149220
1,84.793065,83.537946,81.110245,80.659243,80.668896,80.428571,81.240223,81.221719,80.282805,80.305463,...,80.102222,80.070000,80.722222,83.634038,82.957731,80.192436,80.286986,80.404894,81.100223,80.949889
2,82.084101,82.765125,85.337995,82.790219,77.948344,80.032138,84.727607,91.313295,94.800915,93.752056,...,90.906846,94.022512,91.741897,91.848225,92.620813,89.553571,87.826087,87.740823,89.136819,87.254727
3,84.780488,84.131519,83.511161,83.691537,84.264214,84.814815,88.597534,91.132517,91.035674,89.614699,...,84.612723,85.544036,89.689655,89.909800,87.224944,87.712695,87.341137,87.522854,87.908584,87.799331
4,58.695652,60.945434,66.909800,68.427617,65.987709,66.155481,63.786192,63.081292,63.357461,63.940000,...,65.454849,71.917411,72.933110,67.277592,66.287305,66.215795,66.787068,66.865991,66.048369,65.668521


In [70]:

new_total = total[["id","hr"]]
ex = new_total.groupby("id")["hr"].apply(list).to_frame()


In [71]:
new_ex = ex["hr"].apply(pd.Series)
new_ex.reset_index(inplace=True)




In [72]:
new_ex.head()

,id,0,1,2,3,4,5,6,7,8,...,14,15,16,17,18,19,20,21,22,23
0,1,92.688889,90.442714,91.053393,91.269188,90.459399,91.191111,90.966667,90.596218,90.072303,...,86.011136,83.482720,82.423634,81.283166,80.453229,80.997775,80.985507,79.795987,79.205784,79.149220
1,2,84.793065,83.537946,81.110245,80.659243,80.668896,80.428571,81.240223,81.221719,80.282805,...,80.102222,80.070000,80.722222,83.634038,82.957731,80.192436,80.286986,80.404894,81.100223,80.949889
2,3,82.084101,82.765125,85.337995,82.790219,77.948344,80.032138,84.727607,91.313295,94.800915,...,90.906846,94.022512,91.741897,91.848225,92.620813,89.553571,87.826087,87.740823,89.136819,87.254727
3,4,84.780488,84.131519,83.511161,83.691537,84.264214,84.814815,88.597534,91.132517,91.035674,...,84.612723,85.544036,89.689655,89.909800,87.224944,87.712695,87.341137,87.522854,87.908584,87.799331
4,5,58.695652,60.945434,66.909800,68.427617,65.987709,66.155481,63.786192,63.081292,63.357461,...,65.454849,71.917411,72.933110,67.277592,66.287305,66.215795,66.787068,66.865991,66.048369,65.668521


In [73]:
new_ex = pd.concat([new_ex.id,d],axis=1)

In [74]:
new_ex["y"] =new_ex.id.apply(lambda x: True if x in true_id else False)

In [75]:
all_data = new_ex

In [76]:
new_ex.head()

,id,0,1,2,3,4,5,6,7,8,...,15,16,17,18,19,20,21,22,23,y
0,1,92.688889,90.442714,91.053393,91.269188,90.459399,91.191111,90.966667,90.596218,90.072303,...,83.482720,82.423634,81.283166,80.453229,80.997775,80.985507,79.795987,79.205784,79.149220,False
1,2,84.793065,83.537946,81.110245,80.659243,80.668896,80.428571,81.240223,81.221719,80.282805,...,80.070000,80.722222,83.634038,82.957731,80.192436,80.286986,80.404894,81.100223,80.949889,False
2,3,82.084101,82.765125,85.337995,82.790219,77.948344,80.032138,84.727607,91.313295,94.800915,...,94.022512,91.741897,91.848225,92.620813,89.553571,87.826087,87.740823,89.136819,87.254727,False
3,4,84.780488,84.131519,83.511161,83.691537,84.264214,84.814815,88.597534,91.132517,91.035674,...,85.544036,89.689655,89.909800,87.224944,87.712695,87.341137,87.522854,87.908584,87.799331,False
4,5,58.695652,60.945434,66.909800,68.427617,65.987709,66.155481,63.786192,63.081292,63.357461,...,71.917411,72.933110,67.277592,66.287305,66.215795,66.787068,66.865991,66.048369,65.668521,False


In [77]:
len(true_id)
# len(false_id)
(len(false_id)-4788)/7100*227

77.08408450704226

In [110]:
train = false_id[0:4788] + true_id[0:150]
test = false_id[4788:] + true_id[150:]
# all_data = total.drop(['Sgy', 'MET','eid','tte','death','direct','n_evts','LOS', 'ICU_Pt_Days', 'Mort', 'age', 'race', 'svc'], axis=1)
# col = ['y',"id",'hr', 's2.hr', 's8.hr', 's24.hr', 'n.edrk',
#        'edrk', 's2.edrk', 's8.edrk', 's24.edrk']
# all_data = all_data[col]
X_train = all_data[all_data.id.isin(train)]
x_test = all_data[all_data.id.isin(test)]


In [111]:
len(X_train[X_train.y==True])/len(X_train)


0.030376670716889428

In [112]:
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
scaler = StandardScaler()
y_train = X_train.y
X_train = X_train.drop(['y',"id"], axis=1)

scaler.fit(X_train)
X_train =scaler.transform(X_train)
# new_X_train = X_train
# new_y_train = y_train
sm = SMOTE(random_state=42)
new_X_train, new_y_train = sm.fit_sample(X_train, y_train)

In [113]:
y_test = x_test['y']
test_id = x_test["id"]
x_test = x_test.drop(['y',"id"], axis=1)
x_test = scaler.transform(x_test)

In [114]:
new_X_train = new_X_train.reshape(new_X_train.shape[0], new_X_train.shape[1], 1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], 1)

In [115]:
new_X_train.shape

(9576, 24, 1)

In [94]:
X_train_1 = X_train.reshape(-1,24,1)

In [129]:
input_dim = new_X_train.shape[1]
model= Sequential()
model.add(Conv1D(24, 3, input_shape=new_X_train.shape[1:3]))
model.add(BatchNormalization())
model.add(MaxPooling1D(2))
model.add(Conv1D(48, 3))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.01))
model.add(Dropout(0.2))
model.add(Conv1D(96, 3))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.01))
model.add(Dropout(0.2))
model.add(Conv1D(192, 3))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.01))
model.add(MaxPooling1D(3))
model.add(Dropout(0.2))
model.add(Flatten())
# model.add(LeakyReLU(alpha=0.01))
model.add(Dense(20))

# model.add(Dense(10))
# model.add(LeakyReLU(alpha=0.01))
# model.add(Dropout(0.4))
model.add(Dense(1, activation = 'sigmoid'))

In [130]:
nb_epoch = 30
batch_size = 40
model.compile(optimizer='adam', 
                    loss='binary_crossentropy', 
                    metrics=['accuracy'])
checkpointer = ModelCheckpoint(filepath="model.h5",
                               verbose=0,
                               save_best_only=True)
model.fit(new_X_train, new_y_train,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    shuffle=True,
                    validation_data=(x_test, y_test),
                    verbose=1)

Train on 9576 samples, validate on 2488 samples
Epoch 1/30
9576/9576 [==============================] - 7s 750us/step - loss: 0.7258 - acc: 0.5909 - val_loss: 0.6989 - val_acc: 0.5012
Epoch 2/30
9576/9576 [==============================] - 5s 503us/step - loss: 0.6052 - acc: 0.6741 - val_loss: 0.5117 - val_acc: 0.7086
Epoch 3/30
9576/9576 [==============================] - 5s 502us/step - loss: 0.5539 - acc: 0.7105 - val_loss: 0.6099 - val_acc: 0.6294
Epoch 4/30
9576/9576 [==============================] - 5s 509us/step - loss: 0.5139 - acc: 0.7454 - val_loss: 0.6423 - val_acc: 0.6322
Epoch 5/30
9576/9576 [==============================] - 5s 510us/step - loss: 0.4597 - acc: 0.7833 - val_loss: 0.5461 - val_acc: 0.7046
Epoch 6/30
9576/9576 [==============================] - 5s 507us/step - loss: 0.4234 - acc: 0.8022 - val_loss: 0.4312 - val_acc: 0.7950
Epoch 7/30
9576/9576 [==============================] - 5s 509us/step - loss: 0.3919 - acc: 0.8168 - val_loss: 0.3765 - val_acc: 0.8432


In [131]:
pred = model.predict_proba(x_test)
ind = pred[:,0].argsort()[::-1][:round(len(pred)*0.03)]
test_id = np.array(test_id)

l = list(set(test_id[ind]))
#true patients with event
a = [i for i in l if i in id_true]
len(a)

3

In [139]:
pred[1:10]

array([[8.6996311e-01],
       [2.0188342e-03],
       [6.4646298e-01],
       [2.7167562e-02],
       [3.5692516e-05],
       [1.0522481e-02],
       [6.5564606e-08],
       [6.8482273e-05],
       [5.6745830e-06]], dtype=float32)

In [152]:
# intermediate_layer_model = Model(inputs=model.input,
#                                  outputs=model.layers[14].output)
# intermediate_output = intermediate_layer_model.predict(new_X_train)
# from sklearn.linear_model import LogisticRegression
# clf = LogisticRegression(random_state=0)
# model =clf.fit(intermediate_output,new_y_train)
# test_new_feature = intermediate_layer_model.predict(x_test)
# pred = model.predict_proba(test_new_feature)

In [105]:
# ind = pred[:,1].argsort()[::-1][:round(len(pred)*0.03)]
# test_id = np.array(test_id)
# l = list(set(test_id[ind]))
# #true patients with event
# a = [i for i in l if i in id_true]
# len(a)
pred

array([[0.10647433],
       [0.12488127],
       [0.10372415],
       ...,
       [0.44974664],
       [0.00173322],
       [0.02214571]], dtype=float32)

In [135]:
pred = model.predict(x_test)
sum(y_test[(pred>0.5).reshape(-1,).tolist()])

9

In [136]:
sum(pred>0.5)

array([194])

In [137]:
pre = [False if x <0.5 else True for x in pred ]
from sklearn.metrics import f1_score
f1_score(np.array(y_test), pre)

0.06642066420664207

In [157]:
from sklearn.metrics import precision_recall_curve
precision_recall_curve(np.array(y_test), pred[:,0])

(array([0.03164817, 0.03125   , 0.03126285, ..., 0.        , 0.        ,
        1.        ]),
 array([1.        , 0.98701299, 0.98701299, ..., 0.        , 0.        ,
        0.        ]),
 array([1.1849824e-06, 1.2107510e-06, 1.5183116e-06, ..., 9.8579931e-01,
        9.8893368e-01, 9.9244177e-01], dtype=float32))

In [133]:
from sklearn.metrics import roc_auc_score
roc_auc_score(np.array(y_test), pred)

0.4958819695443502

In [159]:
#misclassified non_event patients
len(list(set([i for i in test_id[ind] if i  in false_id])))

75

In [160]:
#total unique patients 
len(list(set(test_id)))

2488

In [161]:
from random import shuffle
col = all_data.columns.tolist()
z_l= col[1:25]
shuffle(z_l)
col = ["id"] + z_l +["y"]

In [162]:
all_data = all_data[col]

In [163]:
all_data.head()

,id,21,14,13,18,15,4,3,23,10,...,17,8,19,1,7,16,5,22,11,y
0,1,23.919884,94.399771,22.149728,94.053512,22.925676,96.173913,23.056248,24.110815,95.661470,...,23.293479,95.573991,22.999927,23.189782,21.326499,96.183953,22.837611,91.426029,19.652036,False
1,2,10.189456,81.847269,9.437873,80.282536,10.073289,81.100223,9.900825,10.186559,80.971079,...,10.074334,80.498331,9.830293,9.686589,9.727907,80.462736,10.045410,80.456062,9.318443,False
2,3,19.159064,91.244989,18.105199,90.906846,16.130299,91.313295,12.182469,21.788522,89.592423,...,16.721038,93.752056,17.426576,11.350774,18.740759,91.077951,17.096940,91.741897,19.614006,False
3,4,12.959621,84.264214,10.086060,88.597534,10.268548,85.848000,13.027928,13.652618,83.511161,...,10.722505,84.131519,11.788582,12.583667,15.147130,84.814815,14.548634,91.035674,9.486111,False
4,5,12.223723,61.225195,14.365979,60.382386,15.205405,68.016704,14.807263,12.143787,64.357143,...,13.596143,63.094760,12.406882,14.756096,14.816715,59.888641,15.235896,62.145251,13.289369,False


In [164]:



train = false_id[0:4788] + true_id[0:150]
test = false_id[4788:] + true_id[150:]
# all_data = total.drop(['Sgy', 'MET','eid','tte','death','direct','n_evts','LOS', 'ICU_Pt_Days', 'Mort', 'age', 'race', 'svc'], axis=1)
# col = ['y',"id",'hr', 's2.hr', 's8.hr', 's24.hr', 'n.edrk',
#        'edrk', 's2.edrk', 's8.edrk', 's24.edrk']
# all_data = all_data[col]
X_train = all_data[all_data.id.isin(train)]
x_test = all_data[all_data.id.isin(test)]
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
scaler = StandardScaler()
y_train = X_train.y
X_train = X_train.drop(['y',"id"], axis=1)

X_train = scaler.fit_transform(X_train)
# new_X_train = X_train
# new_y_train = y_train
sm = SMOTE(random_state=42)
new_X_train, new_y_train = sm.fit_sample(X_train, y_train)
y_test = x_test['y']
test_id = x_test["id"]
x_test = x_test.drop(['y',"id"], axis=1)
x_test = scaler.fit_transform(x_test)
new_X_train = new_X_train.reshape(new_X_train.shape[0], new_X_train.shape[1], 1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], 1)


In [165]:
input_dim = new_X_train.shape[1]
model= Sequential()
model.add(Conv1D(30, 2, input_shape=new_X_train.shape[1:3]))
model.add(BatchNormalization())
model.add(MaxPooling1D(2))
model.add(Conv1D(30, 2))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.01))
model.add(Dropout(0.2))
model.add(Conv1D(30, 2))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.01))
model.add(Conv1D(20, 2))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.01))
model.add(Dropout(0.2))
model.add(Conv1D(20, 2))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.01))
model.add(MaxPooling1D(2))
model.add(Dropout(0.2))
model.add(Flatten())
# model.add(LeakyReLU(alpha=0.01))
model.add(Dense(10))

# model.add(Dense(10))
# model.add(LeakyReLU(alpha=0.01))
# model.add(Dropout(0.4))
model.add(Dense(1, activation = 'sigmoid'))

In [166]:
nb_epoch = 50
batch_size = 40
model.compile(optimizer='adam', 
                    loss='binary_crossentropy', 
                    metrics=['accuracy'])
checkpointer = ModelCheckpoint(filepath="model.h5",
                               verbose=0,
                               save_best_only=True)
model.fit(new_X_train, new_y_train,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    shuffle=True,
                    validation_data=(x_test, y_test),
                    verbose=1)

Train on 9576 samples, validate on 2488 samples
Epoch 1/50
9576/9576 [==============================] - 4s 436us/step - loss: 0.7021 - acc: 0.5738 - val_loss: 0.7639 - val_acc: 0.3453
Epoch 2/50
9576/9576 [==============================] - 2s 259us/step - loss: 0.6415 - acc: 0.6299 - val_loss: 0.6407 - val_acc: 0.5494
Epoch 3/50
9576/9576 [==============================] - 2s 259us/step - loss: 0.6163 - acc: 0.6560 - val_loss: 0.6329 - val_acc: 0.5599
Epoch 4/50
9576/9576 [==============================] - 2s 252us/step - loss: 0.5985 - acc: 0.6709 - val_loss: 0.7049 - val_acc: 0.4743
Epoch 5/50
9576/9576 [==============================] - 2s 256us/step - loss: 0.5843 - acc: 0.6821 - val_loss: 0.7242 - val_acc: 0.4827
Epoch 6/50
9576/9576 [==============================] - 2s 258us/step - loss: 0.5588 - acc: 0.7060 - val_loss: 0.6515 - val_acc: 0.5812
Epoch 7/50
9576/9576 [==============================] - 2s 254us/step - loss: 0.5459 - acc: 0.7170 - val_loss: 0.6784 - val_acc: 0.5446


In [167]:
pred = model.predict_proba(x_test)
ind = pred[:,0].argsort()[::-1][:round(len(pred)*0.03)]
test_id = np.array(test_id)
l = list(set(test_id[ind]))
#true patients with event
a = [i for i in l if i in id_true]
len(a)

2

In [168]:
roc_auc_score(np.array(y_test), pred[:,0])

0.5286107505103772

In [169]:
sum(y_test[(pred>0.5).reshape(-1,).tolist()])

22

In [170]:
sum(pred>0.5)

array([654])

In [171]:
pre = [False if x <0.5 else True for x in pred ]
from sklearn.metrics import f1_score
f1_score(np.array(y_test), pre)

0.060191518467852256

In [172]:
from sklearn.metrics import precision_recall_curve
precision_recall_curve(np.array(y_test), pred[:,0])

(array([0.03101087, 0.03062047, 0.03063281, ..., 0.        , 0.        ,
        1.        ]),
 array([1.        , 0.98701299, 0.98701299, ..., 0.        , 0.        ,
        0.        ]),
 array([2.91405441e-12, 9.20115893e-12, 1.06033645e-11, ...,
        9.76592243e-01, 9.84086215e-01, 9.93232131e-01], dtype=float32))